In [2]:
import pandas as pd
from dask import dataframe as dd
from dask_ml.linear_model import LinearRegression
from dask_ml.preprocessing import Categorizer, DummyEncoder, MinMaxScaler, StandardScaler
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.metrics import r2_score as r2sk
from dask_ml.metrics import mean_squared_error
from dask_ml.metrics import r2_score
import dask_ml.model_selection as dcv
import dask_ml
from scipy import stats
from distributed import Client, progress
from sklearn.externals.joblib import parallel_backend

In [3]:
client = Client()
client

/Users/marcosberges/anaconda3/lib/python3.6/site-packages/distributed/bokeh/core.py:14: UserWarning: 
Dask needs bokeh >= 0.13.0 for the dashboard.
Continuing without the dashboard.
  "\nDask needs bokeh >= 0.13.0 for the dashboard."


Client Scheduler: tcp://127.0.0.1:57312,Cluster Workers: 4 Cores: 4 Memory: 8.59 GB


In [4]:
ddf=dd.read_csv(
    "https://s3.amazonaws.com/dask-bike/hour.csv?versionId=6G.uFUQD_PJbnvTr4EEbPjYOMQqiEvul",
    sep=",",
    parse_dates=["dteday"])

In [5]:
ddf = ddf.rename(
    columns={
        "weathersit": "weather",
        "mnth": "month",
        "hr": "hour",
        "hum": "humidity",
        "cnt": "count",
        "yr": "year",
    })

In [6]:
ddf = ddf.drop(["dteday"], axis=1)

In [16]:
cats= ["month","hour","holiday","weekday","workingday","weather"]

pipeline = make_pipeline(Categorizer(columns=cats), DummyEncoder(),MinMaxScaler())
ddf_onehot=pipeline.fit_transform(ddf)


### Fix  skewness

In [17]:
def correct_skewness(columns=None, max_skewness=0.5):
    if columns is None:
        raise ValueError(
            f"columns argument is None. Please set columns argument to a list of columns"
        )


    for col in columns:
        skewness = stats.skew(ddf_onehot[col])
        max_val = ddf_onehot[col].max().compute()
        min_val = ddf_onehot[col].min().compute()

        if abs(skewness) > max_skewness and (max_val > 1 or min_val < 0):
            delta = 1.0
            if min_val < 0:
                delta = max(1, -min_val + 1)
            ddf_onehot[col] = da.log(delta + ddf_onehot[col])
    return ddf_onehot


In [18]:
correct_skewness(columns=["season",'instant','year','temp','atemp','humidity','windspeed'])

,instant,season,year,temp,atemp,humidity,windspeed,casual,registered,count,month_1,month_2,month_3,month_4,month_5,month_6,month_7,month_8,month_9,month_10,month_11,month_12,hour_0,hour_1,hour_2,hour_3,hour_4,hour_5,hour_6,hour_7,hour_8,hour_9,hour_10,hour_11,hour_12,hour_13,hour_14,hour_15,hour_16,hour_17,hour_18,hour_19,hour_20,hour_21,hour_22,hour_23,holiday_0,holiday_1,weekday_6,weekday_0,weekday_1,weekday_2,weekday_3,weekday_4,weekday_5,workingday_0,workingday_1,weather_1,weather_2,weather_3,weather_4
npartitions=1,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [10]:
ddf_onehot.head()

,instant,season,year,temp,atemp,humidity,windspeed,casual,registered,count,...,weekday_2,weekday_3,weekday_4,weekday_5,workingday_0,workingday_1,weather_1,weather_2,weather_3,weather_4
0,1,1,0,0.24,0.2879,0.81,0.0,3,13,16,...,0,0,0,0,1,0,1,0,0,0
1,2,1,0,0.22,0.2727,0.80,0.0,8,32,40,...,0,0,0,0,1,0,1,0,0,0
2,3,1,0,0.22,0.2727,0.80,0.0,5,27,32,...,0,0,0,0,1,0,1,0,0,0
3,4,1,0,0.24,0.2879,0.75,0.0,3,10,13,...,0,0,0,0,1,0,1,0,0,0
4,5,1,0,0.24,0.2879,0.75,0.0,0,1,1,...,0,0,0,0,1,0,1,0,0,0


In [19]:
def split_data(dataset, Target):
    X = dataset.loc[:, dataset.columns != Target]
    y = dataset.loc[:, Target]
    train_size = int(len(dataset) * 0.875)
    X_train, X_test, y_train, y_test = (
        X.loc[0:train_size-1], #dask uses loc instead of iloc, and -1 to avoid repeating in both train and test
        X.loc[train_size : len(dataset)],
        y.loc[0:train_size-1], #dask uses loc instead of iloc, and -1 to avoid repeating in both train and test
        y.loc[train_size : len(dataset)],
    )
    return X_train, X_test, y_train, y_test

In [20]:
x_train_reg, x_test_reg, y_train_registered, y_test_registered = split_data(ddf_onehot, "registered")
x_train_casual, x_test_casual, y_train_casual, y_test_casual = split_data(ddf_onehot, "casual")
x_train_count, x_test_count, y_train_count, y_test_count = split_data(ddf_onehot, "count")

## Converting to arrays


The model does not support dask dataframes, so I will try converting all df to dask arrays by using the .values method.

However, we need to drop 2 columns for each of the count, casual and regular; since dask arrays do not have a .drop method, it will be necessary to do it individually and creating 3 different set of arrays.

### Count

In [21]:
X_train_count = x_train_count.drop(["casual", "registered"], axis = 1)
X_test_count = x_test_count.drop(["casual", "registered"], axis = 1)

In [22]:
X_train_count, X_test_count, \
y_train_count, y_test_count = \
X_train_count.values, X_test_count.values, \
y_train_count.values, y_test_count.values

In [ ]:
#pipeline.fit(X_train_count,y_train_count)
#print("training score:",pipeline.score(X_train_count,y_train_count))
#print("test score:",pipeline.score(X_test_count,y_test_count))

In [23]:
lr_params= {'penalty':['l1','l2']}
lr=dcv.GridSearchCV(LinearRegression(),param_grid=lr_params,return_train_score=True)

In [ ]:
#lr.get_params()


In [24]:
with parallel_backend('dask'):
    lr.fit(X_train_count,y_train_count)
    
lr_pred=lr.predict(X_test_count)
lr.score(X_test_count,y_test_count)

ValueError: Multiple constant columns detected!

In [ ]:
def score_lin(X_train, X_test, y_train, y_test):
    lm = LinearRegression()
    lm.fit(X_train, y_train)
    y_pred = lm.predict(X_test)
    print("Intercept:", lm.intercept_)
    print("Coefficients:", lm.coef_)
    print("Mean squared error (MSE): {:.2f}".format(mean_squared_error(y_test, y_pred)))
    print("Variance score (R2): {:.2f}".format(r2sk(y_test.compute(), y_pred.compute())))
    return y_pred

In [ ]:
baseline_count_pred = score_lin(X_train_count, X_test_count, y_train_count, y_test_count)

## Registered

In [ ]:
X_train_registered = x_train_count.drop(["casual", "registered"], axis = 1)
X_test_registered = x_test_count.drop(["casual", "registered"], axis = 1)

In [ ]:
X_train_registered, X_test_registered, \
y_train_registered, y_test_registered = \
X_train_registered.values, X_test_registered.values, \
y_train_registered.values, y_test_registered.values

In [ ]:
baseline_count_pred = score_lin(X_train_registered, X_test_registered, y_train_registered, y_test_registered)